# Project Titan — YOLO Training on Colab

**Objetivo:** Treinar YOLOv8m com dados sintéticos PPPoker-realistas (bordas douradas, showdown, botões)

**Requisitos:**
- Runtime → GPU (T4 gratuita ou melhor)
- Upload dos card assets (52 PNGs de `assets/cards/`)

**Pipeline:**
1. Instalar dependências
2. Upload dos card assets
3. Gerar 5000+ imagens sintéticas PPPoker
4. Treinar YOLOv8m (62 classes)
5. Avaliar e baixar modelo treinado

## 0. Verificar GPU e Instalar Dependências

In [ ]:
# Verificar GPU disponível
!nvidia-smi
import torch
print(f"\nCUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

In [ ]:
# Instalar ultralytics (YOLO) e dependências
!pip install -q ultralytics opencv-python-headless numpy tqdm Pillow

## 1. Clonar Repositório e Configurar Estrutura

In [ ]:
import os

# Clonar repositório
REPO_URL = "https://github.com/jeremiasmarinho/botia.git"
REPO_DIR = "/content/botia"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

PROJECT_DIR = f"{REPO_DIR}/project_titan"
os.chdir(PROJECT_DIR)
print(f"Diretório: {os.getcwd()}")
!ls assets/cards/ | head -10

In [ ]:
# Verificar se os card assets existem
import glob
cards = glob.glob("assets/cards/*.png")
print(f"Card assets encontrados: {len(cards)}/52")

if len(cards) < 52:
    print("\n⚠️  Faltam card assets!")
    print("Opção 1: Verifique se o git clone incluiu os PNGs")
    print("Opção 2: Faça upload manual abaixo")

In [ ]:
# ── UPLOAD MANUAL (só se necessário) ──
# Se os card assets não vieram no clone, faça upload aqui.
# 1. Compacte assets/cards/ em cards.zip no seu PC
# 2. Execute esta célula e faça upload do ZIP

UPLOAD_CARDS = False  # ← Mude para True se precisar fazer upload

if UPLOAD_CARDS:
    from google.colab import files
    print("Faça upload do cards.zip (contendo os 52 PNGs):")
    uploaded = files.upload()
    for fname in uploaded:
        if fname.endswith('.zip'):
            !unzip -o {fname} -d assets/cards/
            !rm {fname}
    cards = glob.glob("assets/cards/*.png")
    print(f"\nCards após upload: {len(cards)}/52")

## 2. Gerar Dados Sintéticos PPPoker-Realistas

In [ ]:
# Configuração da geração
NUM_IMAGES = 5000       # Total de imagens (4250 train + 750 val)
IMGSZ = 640             # Tamanho da imagem
OUTPUT_DIR = "datasets/synthetic_v3"

print(f"Gerando {NUM_IMAGES} imagens PPPoker-realistas...")
print(f"  Gold borders: ✅")
print(f"  Showdown cards: ✅ (25%)")
print(f"  Action buttons: ✅ (50%)")
print(f"  Green table BGs: ✅")
print(f"  Image size: {IMGSZ}x{IMGSZ}")
print(f"  Output: {OUTPUT_DIR}")

In [ ]:
# Gerar! (~3-5 min para 5000 imagens)
!python training/generate_pppoker_data.py \
    --num-images {NUM_IMAGES} \
    --output {OUTPUT_DIR} \
    --imgsz {IMGSZ} \
    --gold-border \
    --showdown \
    --buttons \
    --green-table \
    --seed 42

In [ ]:
# Verificar dados gerados
import os
train_imgs = len(os.listdir(f"{OUTPUT_DIR}/images/train"))
val_imgs = len(os.listdir(f"{OUTPUT_DIR}/images/val"))
train_lbls = len(os.listdir(f"{OUTPUT_DIR}/labels/train"))
val_lbls = len(os.listdir(f"{OUTPUT_DIR}/labels/val"))
print(f"Train: {train_imgs} imagens, {train_lbls} labels")
print(f"Val:   {val_imgs} imagens, {val_lbls} labels")

In [ ]:
# Visualizar amostras geradas
import cv2
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
train_dir = f"{OUTPUT_DIR}/images/train"
samples = sorted(os.listdir(train_dir))[:8]

for ax, fname in zip(axes.flat, samples):
    img = cv2.imread(f"{train_dir}/{fname}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.set_title(fname, fontsize=8)
    ax.axis('off')

plt.suptitle("Amostras PPPoker Sintéticas (com borda dourada)", fontsize=14)
plt.tight_layout()
plt.show()

## 3. Configurar data.yaml para Treino

In [ ]:
# Criar data.yaml otimizado para Colab
# Usa apenas synthetic_v3 (PPPoker-realistic) como fonte principal
# Se titan_cards existir no repo, inclui também

import yaml

DATA_YAML_PATH = "training/data_colab.yaml"

# Definir classes (62 = 52 cartas + 10 UI)
class_names = {
    0: '2c', 1: '2d', 2: '2h', 3: '2s',
    4: '3c', 5: '3d', 6: '3h', 7: '3s',
    8: '4c', 9: '4d', 10: '4h', 11: '4s',
    12: '5c', 13: '5d', 14: '5h', 15: '5s',
    16: '6c', 17: '6d', 18: '6h', 19: '6s',
    20: '7c', 21: '7d', 22: '7h', 23: '7s',
    24: '8c', 25: '8d', 26: '8h', 27: '8s',
    28: '9c', 29: '9d', 30: '9h', 31: '9s',
    32: 'Tc', 33: 'Td', 34: 'Th', 35: 'Ts',
    36: 'Jc', 37: 'Jd', 38: 'Jh', 39: 'Js',
    40: 'Qc', 41: 'Qd', 42: 'Qh', 43: 'Qs',
    44: 'Kc', 45: 'Kd', 46: 'Kh', 47: 'Ks',
    48: 'Ac', 49: 'Ad', 50: 'Ah', 51: 'As',
    52: 'fold', 53: 'check', 54: 'raise',
    55: 'raise_2x', 56: 'raise_2_5x', 57: 'raise_pot',
    58: 'raise_confirm', 59: 'allin',
    60: 'pot', 61: 'stack',
}

# Montar paths
train_paths = [f"{OUTPUT_DIR}/images/train"]
val_paths = [f"{OUTPUT_DIR}/images/val"]

# Incluir titan_cards se existir
if os.path.exists("datasets/titan_cards/images/train"):
    tc_count = len(os.listdir("datasets/titan_cards/images/train"))
    if tc_count > 0:
        train_paths.append("datasets/titan_cards/images/train")
        val_paths.append("datasets/titan_cards/images/val")
        print(f"✅ titan_cards incluído ({tc_count} imagens reais)")

# Incluir synthetic original se existir
if os.path.exists("datasets/synthetic/images/train"):
    sc_count = len(os.listdir("datasets/synthetic/images/train"))
    if sc_count > 0:
        train_paths.append("datasets/synthetic/images/train")
        val_paths.append("datasets/synthetic/images/val")
        print(f"✅ synthetic original incluído ({sc_count} imagens)")

data_config = {
    'path': os.path.abspath('.'),
    'train': train_paths if len(train_paths) > 1 else train_paths[0],
    'val': val_paths if len(val_paths) > 1 else val_paths[0],
    'nc': 62,
    'names': class_names,
}

with open(DATA_YAML_PATH, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False, sort_keys=False)

print(f"\n📄 data.yaml salvo em: {DATA_YAML_PATH}")
print(f"   Train sources: {len(train_paths)}")
print(f"   Val sources:   {len(val_paths)}")
print(f"   Classes: 62 (52 cards + 10 UI)")
!cat {DATA_YAML_PATH}

## 4. Treinar YOLOv8m

**YOLOv8m (Medium)** = sweet spot para o nosso caso:
- 25M parâmetros (vs 3M do Nano atual)
- ~100ms inferência (vs ~200ms do Large)
- Capacidade suficiente para distinguir bordas douradas, cartas sobrepostas PLO6

In [ ]:
# ═══════════════════════════════════════════
# CONFIGURAÇÃO DO TREINO
# ═══════════════════════════════════════════
MODEL_SIZE = "yolov8m.pt"   # Opções: yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt
EPOCHS = 150                # Épocas de treino
BATCH_SIZE = 16             # Batch size (reduzir se OOM)
IMG_SIZE = 640              # Tamanho entrada
PATIENCE = 25               # Early stopping patience
RUN_NAME = "titan_v7_ppk"   # Nome do run

print(f"Modelo:    {MODEL_SIZE}")
print(f"Épocas:    {EPOCHS}")
print(f"Batch:     {BATCH_SIZE}")
print(f"Img size:  {IMG_SIZE}")
print(f"Patience:  {PATIENCE}")
print(f"Run name:  {RUN_NAME}")

In [ ]:
from ultralytics import YOLO

# Carregar modelo base
model = YOLO(MODEL_SIZE)
print(f"\nModelo carregado: {MODEL_SIZE}")
print(f"Parâmetros: {sum(p.numel() for p in model.model.parameters()):,}")

In [ ]:
# ═══════════════════════════════════════════
# TREINAR! (~1-2h com T4, ~30min com A100)
# ═══════════════════════════════════════════
results = model.train(
    data=os.path.abspath(DATA_YAML_PATH),
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    patience=PATIENCE,
    project="runs/detect",
    name=RUN_NAME,
    exist_ok=True,
    # Augmentation (conservador para cartas)
    flipud=0.0,       # Sem flip vertical (cartas têm orientação)
    fliplr=0.0,       # Sem flip horizontal (naipes importam)
    degrees=5.0,      # Rotação leve
    mosaic=1.0,       # Mosaic augmentation
    hsv_h=0.015,      # Variação de hue
    hsv_s=0.4,        # Variação de saturação
    hsv_v=0.3,        # Variação de brilho
    # Learning rate
    lr0=0.01,
    lrf=0.01,
    # Output
    verbose=True,
    plots=True,
)

## 5. Avaliar Resultados

In [ ]:
# Métricas do treino
if results and hasattr(results, 'results_dict'):
    rd = results.results_dict
    print("═" * 50)
    print("MÉTRICAS FINAIS")
    print("═" * 50)
    print(f"  mAP50:      {rd.get('metrics/mAP50(B)', 0):.4f}")
    print(f"  mAP50-95:   {rd.get('metrics/mAP50-95(B)', 0):.4f}")
    print(f"  Precision:  {rd.get('metrics/precision(B)', 0):.4f}")
    print(f"  Recall:     {rd.get('metrics/recall(B)', 0):.4f}")
    print("═" * 50)

In [ ]:
# Visualizar curvas de treino
from IPython.display import Image, display
import os

run_dir = f"runs/detect/{RUN_NAME}"
plots = ["results.png", "confusion_matrix.png", "val_batch0_pred.png", "val_batch1_pred.png"]

for plot_name in plots:
    plot_path = f"{run_dir}/{plot_name}"
    if os.path.exists(plot_path):
        print(f"\n{'─' * 40}")
        print(f"  {plot_name}")
        print(f"{'─' * 40}")
        display(Image(filename=plot_path, width=900))

In [ ]:
# Testar modelo em imagens de validação
best_model = YOLO(f"runs/detect/{RUN_NAME}/weights/best.pt")
print(f"Modelo carregado: {best_model.model.names}")
print(f"Classes: {len(best_model.names)}")
print(f"Parâmetros: {sum(p.numel() for p in best_model.model.parameters()):,}")

# Testar em 4 imagens de validação
val_dir = f"{OUTPUT_DIR}/images/val"
val_images = sorted(os.listdir(val_dir))[:4]

fig, axes = plt.subplots(1, 4, figsize=(24, 6))
for ax, fname in zip(axes, val_images):
    img_path = f"{val_dir}/{fname}"
    results = best_model.predict(img_path, verbose=False, conf=0.25)
    annotated = results[0].plot()
    ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    n_det = len(results[0].boxes) if results[0].boxes is not None else 0
    ax.set_title(f"{fname} ({n_det} detecções)", fontsize=9)
    ax.axis('off')

plt.suptitle("Predições do Modelo Treinado (Validação)", fontsize=14)
plt.tight_layout()
plt.show()

## 6. Exportar e Baixar Modelo

In [ ]:
# Compactar pesos para download
import shutil

weights_dir = f"runs/detect/{RUN_NAME}/weights"
best_pt = f"{weights_dir}/best.pt"
last_pt = f"{weights_dir}/last.pt"

# Tamanho do modelo
best_size = os.path.getsize(best_pt) / (1024 * 1024)
print(f"best.pt: {best_size:.1f} MB")

# Criar ZIP com pesos + data.yaml + métricas
export_dir = f"/content/titan_v7_export"
os.makedirs(export_dir, exist_ok=True)
shutil.copy2(best_pt, f"{export_dir}/best.pt")
shutil.copy2(last_pt, f"{export_dir}/last.pt")
shutil.copy2(DATA_YAML_PATH, f"{export_dir}/data_colab.yaml")

# Copiar plots
for plot_name in ["results.png", "confusion_matrix.png"]:
    src = f"runs/detect/{RUN_NAME}/{plot_name}"
    if os.path.exists(src):
        shutil.copy2(src, f"{export_dir}/{plot_name}")

# ZIP
zip_path = "/content/titan_v7_ppk_weights"
shutil.make_archive(zip_path, 'zip', export_dir)
print(f"\n📦 ZIP criado: {zip_path}.zip ({os.path.getsize(zip_path + '.zip') / 1024 / 1024:.1f} MB)")

In [ ]:
# ═══════════════════════════════════════════
# BAIXAR O MODELO TREINADO
# ═══════════════════════════════════════════
from google.colab import files

# Opção 1: Baixar ZIP completo (pesos + métricas)
files.download(f"{zip_path}.zip")

# Opção 2: Baixar apenas best.pt (descomente se preferir)
# files.download(best_pt)

## 7. (Opcional) Salvar no Google Drive

In [ ]:
# Montar Google Drive e salvar modelo lá
SAVE_TO_DRIVE = False  # ← Mude para True para salvar no Drive

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    drive_dir = "/content/drive/MyDrive/titan_models"
    os.makedirs(drive_dir, exist_ok=True)

    # Copiar modelo
    shutil.copy2(best_pt, f"{drive_dir}/titan_v7_ppk_best.pt")
    shutil.copy2(f"{zip_path}.zip", f"{drive_dir}/titan_v7_ppk_weights.zip")

    print(f"✅ Modelo salvo em: {drive_dir}/")
    !ls -la {drive_dir}/

## 8. Instruções pós-treino

Depois de baixar o `best.pt`:

1. **Copie para o projeto:**
   ```
   Copie best.pt → project_titan/models/titan_v7.pt
   ```

2. **Atualize o config:**
   ```yaml
   # config_club.yaml
   vision:
     yolo_model: models/titan_v7.pt
   ```

3. **Teste com o diagnóstico:**
   ```bash
   python tools/diagnose_vision.py --title "LDPlayer"
   ```

4. **Teste ao vivo:**
   ```bash
   python scripts/live_demo.py --title "LDPlayer"
   ```